## Importando Dependências

In [ ]:
try:
    # Importando a biblioteca do OpenAI
    import openai
    import re
    import json

except ModuleNotFoundError:
    # Baixando a biblioteca do OpenAI caso ela não tenha sido baixada ainda
    !pip install -q openai
    import openai
    import re
    import json

## Informando a chave da API do OpenAI

In [ ]:
openai.api_key = "COLE-SUA-KEY-AQUI"
MODELO = "gpt-3.5-turbo" # Opções: "text-davinci-003", "gpt-3.5-turbo"

## Definindo Informações Básicas para o Algoritmo

In [ ]:
# Define os níveis de dificuldade até chegar ao fim da avaliação
niveis_dificuldade = ["1 - Fácil", "2 - Médio", "3 - Difícil", 'Fim']


# Instrui o algoritmo a sempre iniciar com uma questão de nível fácil
dificuldade = '1 - Fácil'


# Armazena a trajetória de erros e acertos do estudante
trajetoria = []

## Recebendo Informações sobre o Assunto

In [ ]:
contexto = input("Digite o texto tema da aula: ")


prompt_base = f'''
Quero que você atue como um assistente educacional para professores.

Seu objetivo é ler um texto base para uma aula informado pelo professor e criar
questões discursivas para os alunos de acordo com os seguintes níveis de dificuldade:
1 - Fácil
2 - Médio
3 - Difícil

Quando solicitado, você também deve informar o nível de corretude das respostas
informadas de acordo com os números correspondentes às seguintes opções:
1 - Resposta Errada
2 - Resposta Parcialmente Correta
3 - Resposta Correta

O texto base da aula é o seguinte:

"
{contexto}
"
'''

## Fazendo Perguntas Adaptativas para o Aluno

In [ ]:
while dificuldade != "Fim":
    # Monta o texto da pergunta para o ChatGPT
    pergunta_chatgpt = f'''
    {prompt_base}

    De acordo com o contexto acima, gere uma pergunta com nível de dificuldade {dificuldade}.

    Sua resposta deve conter somente o enunciado da pergunta. O nível de dificuldade ou outras informações devem ser omitidas.
    '''
    # Solicita que o ChatGPT gere uma pergunta de acordo com o texto base e o nível de dificuldade selecionados
    prompt_chatgpt = openai.ChatCompletion.create(
        model=MODELO,
        messages=[
            {"role": "user", "content": pergunta_chatgpt},
        ])
    pergunta_gerada = "{}".format(prompt_chatgpt.choices[0]["message"]["content"])


    # Exibe a pergunta gerada e coleta a resposta
    print("\n=============================================================")
    print("=============================================================")
    print("=============================================================")
    print(pergunta_gerada)
    resposta = input("Digite a resposta da pergunta acima: ")



    pergunta_chatgpt = f'''
    {prompt_base}

    O professor apresentou ao aluno a seguinte pergunta:
    "{pergunta_gerada}"

    O aluno informou a seguinte resposta:
    "{resposta}"

    Informe o nível de corretude da resposta do aluno.

    Sua resposta deve necessariamente informar somente o
    número da opção de corretude dentre as opções informadas acima.
    '''
    # Realiza a pergunta sobre a corretude da resposta do aluno ao ChatGPT
    prompt_chatgpt = openai.ChatCompletion.create(
        model=MODELO,
        messages=[
            {"role": "user", "content": pergunta_chatgpt},
        ])


    # Faz o parsing da corretude da resposta do aluno
    corretude_resposta = "{}".format(prompt_chatgpt.choices[0]["message"]["content"])
    numero_corretude_resposta = int(re.findall('[0-9]+', corretude_resposta)[0])


    # Aumenta o nível de dificuldade do aluno caso ele tenha acertado a questão
    if numero_corretude_resposta == 3:
        nova_dificuldade = niveis_dificuldade[niveis_dificuldade.index(dificuldade) + 1]
    else:
        nova_dificuldade = dificuldade


    # Solicita que o ChatGPT analise o humor da resposta do aluno
    pergunta_humor_chatgpt = f'''
    {prompt_base}

    O professor apresentou ao aluno a seguinte pergunta:
    "{pergunta_gerada}"

    O aluno informou a seguinte resposta:
    "{resposta}"

    Analise a resposta fornecida pelo aluno e informe o humor da
    resposta de acordo com os números correspondentes às seguintes opções:
    1. Humor Negativo
    2. Humor Neutro
    3. Humor Positivo
    '''
    prompt_humor_chatgpt = openai.ChatCompletion.create(
        model=MODELO,
        messages=[
            {"role": "user", "content": pergunta_humor_chatgpt},
        ])
    # Faz o parsing do humor da resposta do aluno
    humor_resposta = "{}".format(prompt_humor_chatgpt.choices[0]["message"]["content"])

    # Solicita que o ChatGPT analise o sentimento da resposta do aluno
    pergunta_humor_chatgpt = f'''
    {prompt_base}

    O professor apresentou ao aluno a seguinte pergunta:
    "{pergunta_gerada}"

    O aluno informou a seguinte resposta:
    "{resposta}"

    Analise a resposta fornecida pelo aluno e informe sentimento expressado pela resposta.
    Sua resposta em relação ao sentimento do aluno na resposta deve ser representado em uma única
    palavra (por exemplo: "Confiança", "Insegurança", "Empenho", "Preguiça", "Felicidade", "Alegria",
    "Entusiasmo", "Gratidão", "Esperança", "Satisfação", "Amor", "Tristeza", "Raiva", "Frustração",
    "Medo", "Decepção", "Culpa", "Ansiedade", etc.).
    '''
    prompt_sentimento_chatgpt = openai.ChatCompletion.create(
        model=MODELO,
        messages=[
            {"role": "user", "content": pergunta_humor_chatgpt},
        ])
    # Faz o parsing do humor da resposta do aluno
    sentimento_resposta = "{}".format(prompt_sentimento_chatgpt.choices[0]["message"]["content"])


    # Armazena o progresso do aluno
    trajetoria.append(
        {
             "dificuldade": dificuldade,
             "corretude": corretude_resposta,
             "humor": humor_resposta,
             "sentimento": sentimento_resposta,
        }
    )

    # Atualiza a dificuldade do aluno
    dificuldade = nova_dificuldade


In [ ]:
print("=============================")
print("==== TRAJETÓRIA DO ALUNO ====")
print("=============================")

print(f"Número de Perguntas: {len(trajetoria)}\n")

for numero_pergunta, resposta in enumerate(trajetoria, 1):
    dificuldade = resposta["dificuldade"]
    corretude = resposta["corretude"]
    humor = resposta["humor"]
    sentimento = resposta["sentimento"]

    print(f"=== Pergunta {numero_pergunta}")
    print(f"\tDificuldade: {dificuldade}")
    print(f"\tCorretude: {corretude}")
    print(f"\tHumor: {humor}")
    print(f"\tSentimento: {sentimento}")
    
    print("\n")